Ce code marche sur google colab mais pas sur mon VSCode. 

In [18]:
!pip install "gym[atari, accept-rom-license]" # lancer cette ligne a chaque fois qu'on commence une session
# !pip install setuptools==65.5.0 "wheel<0.40.0" # wheel 0.38 pour moi
# probleme avec l'installation de gym 0.21.0, il faut changer la version de setuptools et lancer : !pip install setuptools==65.5.0 "wheel<0.40.0" avant de lancer la commande ci dessous
# !pip install gym==0.21.0
# !pip list

In [19]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Change according to your own Google Drive config
!cp -r drive/MyDrive/test_bon_jeu/* .

Mounted at /content/drive


In [20]:
%load_ext autoreload
%autoreload 2

In [21]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder

In [30]:
from collections import namedtuple
from itertools import count
import math
import random
import numpy as np
import time

import gym

from wrappers import *
from memory import ReplayMemory
from models import *

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

Transition = namedtuple('Transion',
                        ('state', 'action', 'next_state', 'reward'))


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END)* \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # return policy_net(state.to('cuda')).max(1)[1].view(1,1)
            return policy_net(state).max(1)[1].view(1,1)
    else:
        return torch.tensor([[random.randrange(4)]], device=device, dtype=torch.long)


def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    """
    zip(*transitions) unzips the transitions into
    Transition(*) creates new named tuple
    batch.state - tuple of all the states (each state is a tensor)
    batch.next_state - tuple of all the next states (each state is a tensor)
    batch.reward - tuple of all the rewards (each reward is a float)
    batch.action - tuple of all the actions (each action is an int)
    """
    batch = Transition(*zip(*transitions))

    actions = tuple((map(lambda a: torch.tensor([[a]], device='cuda'), batch.action)))
    rewards = tuple((map(lambda r: torch.tensor([r], device='cuda'), batch.reward)))

    non_final_mask = torch.tensor(
        tuple(map(lambda s: s is not None, batch.next_state)),
        device=device, dtype=torch.bool) # changmeent uint8 to bool

    non_final_next_states = torch.cat([s for s in batch.next_state
                                       if s is not None]).to('cuda')


    state_batch = torch.cat(batch.state).to('cuda')
    action_batch = torch.cat(actions)
    reward_batch = torch.cat(rewards)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

def get_state(obs):
    state = np.array(obs)
    state = state.transpose((2, 0, 1))
    state = torch.from_numpy(state)
    return state.unsqueeze(0)

def train(env, n_episodes, render=False):
    for episode in range(n_episodes):
        obs = env.reset()
        state = get_state(obs)
        total_reward = 0.0
        for t in count():
            action = select_action(state)

            if render:
                env.render()

            obs, reward, done, info = env.step(action)

            total_reward += reward

            if not done:
                next_state = get_state(obs)
            else:
                next_state = None

            reward = torch.tensor([reward], device=device)

            memory.push(state, action.to('cpu'), next_state, reward.to('cpu'))
            state = next_state

            if steps_done > INITIAL_MEMORY:
                optimize_model()

                if steps_done % TARGET_UPDATE == 0:
                    target_net.load_state_dict(policy_net.state_dict())

            if done:
                break
        if episode % 20 == 0:
                print('Total steps: {} \t Episode: {}/{} \t Total reward: {}'.format(steps_done, episode, t, total_reward))
    env.close()
    return

def test(env, n_episodes, policy, render=True):
    #env = gym.wrappers.Monitor(env, './DemonAttack_video/' + 'demon_attack_video')
    for episode in range(n_episodes):
        obs = env.reset()
        state = get_state(obs)
        total_reward = 0.0
        for t in count():
            #action = policy(state.to('cuda')).max(1)[1].view(1,1)
            action = policy(state).max(1)[1].view(1,1)

            if render:
                env.render()
                time.sleep(0.02)

            obs, reward, done, info = env.step(action)

            total_reward += reward

            if not done:
                next_state = get_state(obs)
            else:
                next_state = None

            state = next_state

            if done:
                print("Finished Episode {} with reward {}".format(episode, total_reward))
                break

    env.close()
    return

if __name__ == '__main__':
    # set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # hyperparameters
    BATCH_SIZE = 32
    GAMMA = 0.99
    EPS_START = 1
    EPS_END = 0.02
    EPS_DECAY = 1000000
    TARGET_UPDATE = 1000
    RENDER = 'Human'
    lr = 1e-4
    INITIAL_MEMORY = 10000
    MEMORY_SIZE = 10 * INITIAL_MEMORY

    # create networks
    policy_net = DQN(n_actions=4).to(device)
    target_net = DQN(n_actions=4).to(device)
    target_net.load_state_dict(policy_net.state_dict())

    # setup optimizer
    optimizer = optim.Adam(policy_net.parameters(), lr=lr)

    steps_done = 0

    # create environment
    env = gym.make("DemonAttack-v4")
    env = make_env(env)

    # initialize replay memory
    memory = ReplayMemory(MEMORY_SIZE)

    # moi
    # before_training = "before_training.mp4"
    before_training = f'DemonAttack_video_{int(time.time())}.mp4'
    video = VideoRecorder(env, before_training)

    video_dir = f'./data_DemonAttack_video_{int(time.time())}'

    env.reset()
    for i in range (200):
      video.capture_frame()
      action = env.action_space.sample()
      obs, reward, done, info = env.step(action)
      env.render("rgb_array")

    video.close()
    #env.close()
    #env = gym.wrappers.Monitor(env, video_dir)
    # train model
    train(env, 400)
    torch.save(policy_net, "demonattack_model")
    policy_net = torch.load("demonattack_model")
    test(env, 1, policy_net, render=False)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:78: DeprecationWarning: WARN: Recording ability for environment DemonAttack-v4 initialized with `render_mode=None` is marked as deprecated and will be removed in the future.
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:101: DeprecationWarning: WARN: <class 

Total steps: 35 	 Episode: 0/34 	 Total reward: 10.0
Total steps: 1205 	 Episode: 20/29 	 Total reward: 20.0
Total steps: 1898 	 Episode: 40/120 	 Total reward: 40.0
Total steps: 3102 	 Episode: 60/14 	 Total reward: 20.0
Total steps: 3878 	 Episode: 80/58 	 Total reward: 20.0
Total steps: 4948 	 Episode: 100/12 	 Total reward: 0.0
Total steps: 6302 	 Episode: 120/7 	 Total reward: 0.0
Total steps: 7516 	 Episode: 140/5 	 Total reward: 0.0
Total steps: 8323 	 Episode: 160/38 	 Total reward: 10.0
Total steps: 9363 	 Episode: 180/289 	 Total reward: 90.0


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [23]:
test(env, 1, policy_net, render=False)

Finished Episode 0 with reward 0.0


In [ ]:
# from gym.wrappers.monitoring.video_recorder import VideoRecorder

# before_training = "before_training.mp4"

# video = VideoRecorder(env, before_training)
# env.reset()
# for i in range (200):
#   video.capture_frame()
#   action = env.action_space.sample()
#   obs, reward, done, info = env.step(action)
#   env.render()

# video.close()
# env.close()

Error: render(mode='human') is deprecated. Please supply `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). The new `render_mode` keyword argument supports DPI scaling, audio, and native framerates.